In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
train_df = pd.read_csv('train_df_version2.csv')
train_df.head(15)

,county_state,0,1,2,3,4,5,6,7,8,...,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,state
0,Abbeville County South Carolina,1.335653,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,...,1.8,1.6,2.0,1.5,35254.0,36685.0,38741.0,43090.0,45710.0,South Carolina
1,Acadia Parish Louisiana,1.695543,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,...,1.7,2.1,2.5,1.5,40492.0,41177.0,43396.0,44412.0,42368.0,Louisiana
2,Accomack County Virginia,2.910939,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,...,0.6,0.7,0.8,0.3,42260.0,43210.0,46073.0,46178.0,50601.0,Virginia
3,Ada County Idaho,11.778187,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,...,2.1,1.9,2.0,2.0,60151.0,63137.0,66293.0,69952.0,75115.0,Idaho
4,Adair County Iowa,2.782208,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,...,0.8,0.6,0.4,0.3,49477.0,51859.0,53363.0,55700.0,57944.0,Iowa
5,Adair County Kentucky,2.069413,1.972105,1.777489,1.777489,1.783977,1.787572,1.742070,1.696568,1.716069,...,1.2,1.1,0.8,0.6,36575.0,39346.0,38021.0,43026.0,46652.0,Kentucky
6,Adair County Missouri,2.796565,2.916487,2.916487,2.964455,2.964455,2.976820,2.923920,2.962393,3.015293,...,2.5,2.0,1.7,1.7,38750.0,40046.0,41929.0,42301.0,46639.0,Missouri
7,Adair County Oklahoma,0.785999,0.761437,0.749156,0.798281,0.792140,0.792675,0.743517,0.731228,0.755807,...,0.3,0.2,0.2,0.2,33366.0,32986.0,34695.0,34375.0,37940.0,Oklahoma
8,Adams County Colorado,5.505571,5.445589,5.453511,5.442194,5.432857,5.327217,5.248416,5.231274,5.308692,...,2.8,2.7,3.0,2.7,64087.0,67575.0,71202.0,73817.0,78304.0,Colorado
9,Adams County Idaho,2.889640,2.920381,2.920381,2.951122,3.043344,3.107861,3.138330,3.046923,3.138330,...,0.4,0.0,0.0,0.6,42727.0,45319.0,48856.0,50309.0,52746.0,Idaho


In [3]:
train_df.isnull().sum().sum()

0

In [4]:
from lightgbm import LGBMRegressor

In [5]:
train_df.columns

Index(['county_state', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', 'cfips', 'pct_bb_2017', 'pct_bb_2018',
       'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017',
       'pct_college_2018', 'pct_college_2019', 'pct_college_2020',
       'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018',
       'pct_foreign_born_2019', 'pct_foreign_born_2020',
       'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018',
       'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021',
       'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019',
       'median_hh_inc_2020', 'median_hh_inc_2021', 'state'],
      dtype='object')

In [10]:
lookup_map = dict(zip(list(train_df.index), train_df['cfips'].values))
#lookup_map

In [11]:
train_df_dummies = pd.get_dummies(train_df, columns=['state'], drop_first=True)
train_df_dummies.shape

(3135, 116)

In [12]:
train_df_dummies = train_df_dummies.drop(columns=['county_state', 'cfips'], axis=1)

In [14]:
X = train_df_dummies.drop(columns=['37', '38'], axis=1)
y1 = train_df_dummies['37']
y2 = train_df_dummies['38']

In [15]:
lgbm_model = LGBMRegressor()

lgbm_model.fit(X, y1)

preds1 = lgbm_model.predict(X)

print(np.mean((preds1 - y1.values)**2))

3.381519349334474


In [16]:
from xgboost import XGBRegressor

/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [17]:
xgb_model = XGBRegressor()

xgb_model.fit(X, y1)

preds1b = xgb_model.predict(X)

print(np.mean((preds1b - y1.values)**2))

/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.0004454491256079961


In [21]:
y1.values.max()

128.26224

In [22]:
preds1.max()

60.56948164644332

In [23]:
preds1b.max()

128.23438

In [24]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.335653,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,...,0,0,0,0,0,0,0,0,0,0
1,1.695543,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,...,0,0,0,0,0,0,0,0,0,0
2,2.910939,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,...,0,0,0,0,0,1,0,0,0,0
3,11.778187,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,...,0,0,0,0,0,0,0,0,0,0
4,2.782208,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_cols_original = X.columns

X['37'] = preds1
X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming,37
0,1.335653,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,...,0,0,0,0,0,0,0,0,0,1.324949
1,1.695543,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,...,0,0,0,0,0,0,0,0,0,1.700061
2,2.910939,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,...,0,0,0,0,1,0,0,0,0,2.405701
3,11.778187,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,...,0,0,0,0,0,0,0,0,0,12.355765
4,2.782208,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,...,0,0,0,0,0,0,0,0,0,2.092575


In [27]:
X = X.drop(columns=['0'], axis=1)

ye = dict(zip([str(i) for i in range(1, 38)], [str(i-1) for i in range(1, 38)]))

X = X.rename(columns=ye)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming,36
0,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,1.236341,...,0,0,0,0,0,0,0,0,0,1.324949
1,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,1.655730,...,0,0,0,0,0,0,0,0,0,1.700061
2,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,2.799583,...,0,0,0,0,1,0,0,0,0,2.405701
3,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,11.766225,...,0,0,0,0,0,0,0,0,0,12.355765
4,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,3.099803,...,0,0,0,0,0,0,0,0,0,2.092575


In [28]:
X = X[X_cols_original]
X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,1.236341,...,0,0,0,0,0,0,0,0,0,0
1,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,1.655730,...,0,0,0,0,0,0,0,0,0,0
2,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,2.799583,...,0,0,0,0,0,1,0,0,0,0
3,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,11.766225,...,0,0,0,0,0,0,0,0,0,0
4,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,3.099803,...,0,0,0,0,0,0,0,0,0,0


In [29]:
preds2 = lgbm_model.predict(X)

print(np.mean((preds2 - y2.values)**2))

12.161372479303648


In [33]:
preds2.max()

59.044996082098194

In [32]:
y2.values.max()

227.75665

In [34]:
X = train_df_dummies.drop(columns=['37', '38'], axis=1)
y1 = train_df_dummies['37']
y2 = train_df_dummies['38']

In [35]:
X_cols_original = X.columns

X['37'] = preds1b

X = X.drop(columns=['0'], axis=1)

ye = dict(zip([str(i) for i in range(1, 38)], [str(i-1) for i in range(1, 38)]))

X = X.rename(columns=ye)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming,36
0,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,1.236341,...,0,0,0,0,0,0,0,0,0,1.337017
1,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,1.655730,...,0,0,0,0,0,0,0,0,0,1.682382
2,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,2.799583,...,0,0,0,0,1,0,0,0,0,2.433113
3,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,11.766225,...,0,0,0,0,0,0,0,0,0,12.158493
4,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,3.099803,...,0,0,0,0,0,0,0,0,0,2.082283


In [36]:
X = X[X_cols_original]
X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,1.236341,...,0,0,0,0,0,0,0,0,0,0
1,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,1.655730,...,0,0,0,0,0,0,0,0,0,0
2,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,2.799583,...,0,0,0,0,0,1,0,0,0,0
3,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,11.766225,...,0,0,0,0,0,0,0,0,0,0
4,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,3.099803,...,0,0,0,0,0,0,0,0,0,0


In [37]:
preds2b = xgb_model.predict(X)

print(np.mean((preds2b - y2.values)**2))

3.314576912668769


/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [42]:
preds2b.min()

0.02428633

In [40]:
y2.values.max()

227.75665

In [46]:
X = train_df_dummies.drop(columns=['38'], axis=1)
y = train_df_dummies['38']

xgb_model_final = XGBRegressor()

xgb_model_final.fit(X, y)

/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [47]:
X_cols_original = X.columns

X = X.drop(columns=['0'], axis=1)

X['38'] = y.values

ye = dict(zip([str(i) for i in range(1, 39)], [str(i-1) for i in range(1, 39)]))

X = X.rename(columns=ye)

X = X[X_cols_original]

X.head()

,0,1,2,3,4,5,6,7,8,9,...,state_South Dakota,state_Tennessee,state_Texas,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,1.197482,1.258892,1.228187,1.217952,1.195301,1.149131,1.210691,1.205561,1.215821,1.236341,...,0,0,0,0,0,0,0,0,0,0
1,1.651956,1.651956,1.643239,1.721696,1.690496,1.686150,1.675286,1.690496,1.662249,1.655730,...,0,0,0,0,0,0,0,0,0,0
2,2.706865,2.833930,2.968696,2.934042,2.888519,2.784115,2.807316,2.842118,2.830517,2.799583,...,0,0,0,0,0,1,0,0,0,0
3,11.920884,11.923940,12.011331,11.977107,11.650465,11.528472,11.595257,11.701518,11.743073,11.766225,...,0,0,0,0,0,0,0,0,0,0
4,2.746766,2.782208,2.870813,2.853092,2.974377,2.956459,3.010213,3.010213,3.010213,3.099803,...,0,0,0,0,0,0,0,0,0,0


In [48]:
monthly_predictions = {}

for idx in range(1, 9):
    
    preds = xgb_model_final.predict(X)
    
    monthly_predictions[idx] = preds
    
    X_cols_original = X.columns

    X = X.drop(columns=['0'], axis=1)

    X['38'] = preds

    ye = dict(zip([str(i) for i in range(1, 39)], [str(i-1) for i in range(1, 39)]))

    X = X.rename(columns=ye)

    X = X[X_cols_original]
    
    

/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/alecclarkfeather/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:262

In [49]:
monthly_predictions

{1: array([1.2706015 , 1.6852105 , 2.4750156 , ..., 0.67304647, 0.4348624 ,
        0.30942717], dtype=float32),
 2: array([1.290952  , 1.6852105 , 2.4750156 , ..., 0.67304647, 0.4348624 ,
        0.27851108], dtype=float32),
 3: array([1.290952  , 1.6852105 , 2.4750156 , ..., 0.67304647, 0.4348624 ,
        0.27851108], dtype=float32),
 4: array([1.2933019 , 1.6710843 , 2.4774027 , ..., 0.67304647, 0.44359258,
        0.27851108], dtype=float32),
 5: array([1.2933019 , 1.6826999 , 2.4774027 , ..., 0.67304647, 0.44359258,
        0.27851108], dtype=float32),
 6: array([1.2933019 , 1.6826999 , 2.4774027 , ..., 0.67304647, 0.43732193,
        0.27851108], dtype=float32),
 7: array([1.2933019 , 1.677762  , 2.4774027 , ..., 0.67304647, 0.43732193,
        0.27851108], dtype=float32),
 8: array([1.3110579 , 1.6826999 , 2.4774027 , ..., 0.67304647, 0.43732193,
        0.27314848], dtype=float32)}

In [50]:
monthly_predictions[1].max(), monthly_predictions[1].min(), monthly_predictions[1].mean()

(227.68207, 0.020723669, 4.0217147)

In [51]:
monthly_predictions[2].max(), monthly_predictions[2].min(), monthly_predictions[2].mean()

(227.68207, 0.020723669, 4.03669)

In [52]:
monthly_predictions[8].max(), monthly_predictions[8].min(), monthly_predictions[8].mean()

(227.68207, 0.020723669, 4.1255107)

In [55]:
codes = [lookup_map[i] for i in range(len(train_df_dummies))]
#codes

In [57]:
submit1 = dict(zip(codes, list(monthly_predictions[1])))
submit2 = dict(zip(codes, list(monthly_predictions[2])))
submit3 = dict(zip(codes, list(monthly_predictions[3])))
submit4 = dict(zip(codes, list(monthly_predictions[4])))
submit5 = dict(zip(codes, list(monthly_predictions[5])))
submit6 = dict(zip(codes, list(monthly_predictions[6])))
submit7 = dict(zip(codes, list(monthly_predictions[7])))
submit8 = dict(zip(codes, list(monthly_predictions[8])))

In [59]:
df1 = pd.DataFrame(submit1.items())
df1.columns = ['cfips', 'pred']
df1

,cfips,pred
0,45001.0,1.270602
1,22001.0,1.685210
2,51001.0,2.475016
3,16001.0,12.052652
4,19001.0,2.169907
...,...,...
3130,4027.0,2.238080
3131,8125.0,3.147171
3132,48505.0,0.673046
3133,48507.0,0.434862


In [60]:
df1['first_day_of_month'] = '2022-11-01'

In [61]:
df2 = pd.DataFrame(submit2.items())
df2.columns = ['cfips', 'pred']
df2['first_day_of_month'] = '2022-12-01'

df3 = pd.DataFrame(submit3.items())
df3.columns = ['cfips', 'pred']
df3['first_day_of_month'] = '2023-01-01'

df4 = pd.DataFrame(submit4.items())
df4.columns = ['cfips', 'pred']
df4['first_day_of_month'] = '2023-02-01'

df5 = pd.DataFrame(submit5.items())
df5.columns = ['cfips', 'pred']
df5['first_day_of_month'] = '2023-03-01'

df6 = pd.DataFrame(submit6.items())
df6.columns = ['cfips', 'pred']
df6['first_day_of_month'] = '2023-04-01'

df7 = pd.DataFrame(submit7.items())
df7.columns = ['cfips', 'pred']
df7['first_day_of_month'] = '2023-05-01'

df8 = pd.DataFrame(submit8.items())
df8.columns = ['cfips', 'pred']
df8['first_day_of_month'] = '2023-06-01'

In [62]:
df1['cfips'] = df1['cfips'].astype(int)
df2['cfips'] = df2['cfips'].astype(int)
df3['cfips'] = df3['cfips'].astype(int)
df4['cfips'] = df4['cfips'].astype(int)
df5['cfips'] = df5['cfips'].astype(int)
df6['cfips'] = df6['cfips'].astype(int)
df7['cfips'] = df7['cfips'].astype(int)
df8['cfips'] = df8['cfips'].astype(int)

In [63]:
df1_sorted = df1.sort_values(by='cfips')
df2_sorted = df2.sort_values(by='cfips')
df3_sorted = df3.sort_values(by='cfips')
df4_sorted = df4.sort_values(by='cfips')
df5_sorted = df5.sort_values(by='cfips')
df6_sorted = df6.sort_values(by='cfips')
df7_sorted = df7.sort_values(by='cfips')
df8_sorted = df8.sort_values(by='cfips')

In [64]:
submission_df = pd.concat([df1_sorted, df2_sorted, df3_sorted, df4_sorted, df5_sorted, df6_sorted,
                          df7_sorted, df8_sorted], axis=0, ignore_index=True)
submission_df

,cfips,pred,first_day_of_month
0,1001,3.492666,2022-11-01
1,1003,8.335525,2022-11-01
2,1005,1.185745,2022-11-01
3,1007,1.287833,2022-11-01
4,1009,1.852226,2022-11-01
...,...,...,...
25075,56037,2.831048,2023-06-01
25076,56039,30.019972,2023-06-01
25077,56041,4.052368,2023-06-01
25078,56043,3.061377,2023-06-01


In [65]:
submission_df['row_id'] = submission_df['cfips'].astype(str) + '_' + submission_df['first_day_of_month']
submission_df

,cfips,pred,first_day_of_month,row_id
0,1001,3.492666,2022-11-01,1001_2022-11-01
1,1003,8.335525,2022-11-01,1003_2022-11-01
2,1005,1.185745,2022-11-01,1005_2022-11-01
3,1007,1.287833,2022-11-01,1007_2022-11-01
4,1009,1.852226,2022-11-01,1009_2022-11-01
...,...,...,...,...
25075,56037,2.831048,2023-06-01,56037_2023-06-01
25076,56039,30.019972,2023-06-01,56039_2023-06-01
25077,56041,4.052368,2023-06-01,56041_2023-06-01
25078,56043,3.061377,2023-06-01,56043_2023-06-01


In [66]:
submission_final = submission_df[['row_id', 'pred']]
submission_final

,row_id,pred
0,1001_2022-11-01,3.492666
1,1003_2022-11-01,8.335525
2,1005_2022-11-01,1.185745
3,1007_2022-11-01,1.287833
4,1009_2022-11-01,1.852226
...,...,...
25075,56037_2023-06-01,2.831048
25076,56039_2023-06-01,30.019972
25077,56041_2023-06-01,4.052368
25078,56043_2023-06-01,3.061377


In [67]:
submission_final.columns = ['row_id', 'microbusiness_density']
submission_final

,row_id,microbusiness_density
0,1001_2022-11-01,3.492666
1,1003_2022-11-01,8.335525
2,1005_2022-11-01,1.185745
3,1007_2022-11-01,1.287833
4,1009_2022-11-01,1.852226
...,...,...
25075,56037_2023-06-01,2.831048
25076,56039_2023-06-01,30.019972
25077,56041_2023-06-01,4.052368
25078,56043_2023-06-01,3.061377


In [68]:
submission_final.to_csv('godaddy_second_submission.csv', index=False)